# Tratando Dados Ausentes

A diferença entre os dados encontrados em muitos tutoriais e os dados no mundo real é que os dados do mundo real raramente são limpos e homogêneos. Em particular, muitos conjuntos de dados interessantes terão alguma quantidade de dados ausentes. Para tornar as coisas ainda mais complicadas, fontes de dados diferentes podem indicar dados ausentes de maneiras diferentes.

Nesta notebook, será apresentado considerações gerais sobre dados ausentes, discutiremos como o Pandas escolhe representá-los e demonstraremos algumas ferramentas internas do Pandas para lidar com dados ausentes no Python. Aqui e ao longo do livro, nos referiremos a dados ausentes em geral como valores *null*, *NaN* ou *NA* values.

## Trade-Offs em convenções de dados ausentes

Há vários esquemas que foram desenvolvidos para indicar a presença de dados ausentes em uma tabela ou DataFrame. Geralmente, eles giram em torno de uma das duas estratégias: usar uma máscara que indique globalmente valores ausentes ou escolher um valor sentinela que indique uma entrada ausente.

Na abordagem de mascaramento, a máscara pode ser uma matriz booleana totalmente separada ou pode envolver a apropriação de um bit na representação de dados para indicar localmente o status nulo de um valor.

Na abordagem sentinela, o valor sentinela poderia ser uma convenção específica de dados, como indicar um valor inteiro ausente com -9999 ou algum padrão de bits raro, ou poderia ser uma convenção mais global, como indicar um valor de ponto flutuante ausente com NaN (não é um número), um valor especial que faz parte da especificação de ponto flutuante IEEE.

Nenhuma dessas abordagens é isenta de compromisso: o uso de uma matriz de máscara separada requer a alocação de uma matriz booleana adicional, que adiciona sobrecarga no armazenamento e na computação. Um valor sentinela reduz o intervalo de valores válidos que podem ser representados e pode exigir lógica extra (geralmente não otimizada) na aritmética da CPU e da GPU. Valores especiais comuns como NaN não estão disponíveis para todos os tipos de dados.

Como na maioria dos casos em que não existe uma escolha ideal universalmente, idiomas e sistemas diferentes usam convenções diferentes. Por exemplo, a linguagem R usa padrões de bits reservados em cada tipo de dados como valores sentinela indicando dados ausentes, enquanto o sistema SciDB usa um byte extra anexado a cada célula que indica um estado NA.

## Dados ausentes no Pandas

A maneira como o Pandas lida com os valores ausentes é restringida por sua dependência do pacote NumPy, que não possui uma noção interna de valores de NA para tipos de dados de ponto não flutuante.

Os pandas poderiam ter seguido o exemplo de R na especificação de padrões de bits para cada tipo de dados individual para indicar nulidade, mas essa abordagem acaba sendo bastante difícil. Enquanto R contém quatro tipos de dados básicos, o NumPy suporta muito mais do que isso: por exemplo, enquanto R possui um único tipo inteiro, o NumPy suporta catorze tipos inteiros básicos depois que você considera as precisões, assinaturas e endianidades disponíveis da codificação. A reserva de um padrão de bits específico em todos os tipos de NumPy disponíveis levaria a uma sobrecarga pesada em operações especiais de caixa especial para vários tipos, provavelmente exigindo até uma nova bifurcação do pacote NumPy. Além disso, para os tipos de dados menores (como números inteiros de 8 bits), sacrificar um pouco para usar como máscara reduzirá significativamente o intervalo de valores que ele pode representar.

O NumPy tem suporte para matrizes mascaradas - ou seja, matrizes que possuem uma matriz de máscara booleana separada anexada para marcar os dados como "bons" ou "ruins". O Pandas poderia ter derivado disso, mas a sobrecarga de armazenamento, computação e manutenção de código faz dessa uma opção pouco atraente.

Com essas restrições em mente, o Pandas optou por usar sentinelas para dados ausentes e, além disso, optou por usar dois valores nulos do Python já existentes: o valor `NaN` de ponto flutuante especial e o objeto `None` do Python. Essa escolha tem alguns efeitos colaterais, como veremos, mas na prática acaba sendo um bom compromisso na maioria dos casos de interesse.

## None: dados ausentes

O primeiro valor sentinela usado pelo Pandas é `None`, um objeto singleton do Python que geralmente é usado para a falta de dados no código Python. Por ser um objeto Python, o `None` não pode ser usado em nenhuma matriz NumPy/Pandas arbitrária, mas apenas em matrizes com o tipo de dados `'objeto'` (ou seja, matrizes de objetos Python):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Esse `dtype = object` significa que a melhor representação de tipo comum que NumPy poderia inferir para o conteúdo da matriz é que eles são objetos Python. Embora esse tipo de matriz de objetos seja útil para alguns propósitos, quaisquer operações nos dados serão realizadas no nível Python, com muito mais sobrecarga do que as operações tipicamente rápidas vistas para matrizes com tipos nativos:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
49.3 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.5 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



O uso de objetos Python em uma matriz também significa que, se você realizar operações como `sum()` ou `min()` em uma matriz com o valor `None`, geralmente ocorrerá um erro:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## NaN: Valores numéricos ausentes

A outra representação de dados ausentes, `NaN` (sigla para Not a Number), é diferente; é um valor especial de ponto flutuante reconhecido por todos os sistemas que usam a representação padrão de ponto flutuante IEEE:

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Observe que o `NumPy` escolheu um tipo de ponto flutuante nativo para esta matriz: isso significa que, diferentemente da matriz de objetos de antes, essa matriz suporta operações rápidas inseridas no código compilado. Você deve estar ciente de que o `NaN` é um pouco como um vírus de dados - ele infecta qualquer outro objeto em que toca. Independentemente da operação, o resultado da aritmética com `NaN` será outro `NaN`:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

Observe que isso significa que as agregações sobre os valores estão bem definidas (ou seja, não resultam em erro), mas nem sempre são úteis:

In [8]:
vals2.sum(), vals2.min(), vals2.max()

/home/jailsonpereira/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/home/jailsonpereira/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


(nan, nan, nan)

O `NumPy` fornece algumas agregações especiais que ignoram esses valores ausentes:

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## NaN e None no Pandas

`NaN` e `None` têm o seu lugar, e o Pandas é construído para lidar com os dois quase de forma intercambiável, convertendo entre eles quando apropriado:

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para tipos que não têm um valor sentinela disponível, o Pandas faz a conversão automática de tipos quando os valores de NA estão presentes. Por exemplo, se definirmos um valor em uma matriz inteira como `np.nan`, ele será automaticamente convertido para um tipo de ponto flutuante para acomodar o `NA`:

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

## Operações com valores Null

Como vimos, o Pandas trata `None` e `NaN` como essencialmente intercambiáveis por indicar valores ausentes ou nulos. Para facilitar essa convenção, existem vários métodos úteis para detectar, remover e substituir valores nulos nas estruturas de dados do Pandas. Eles são:

- `isnull()`: Gera uma máscara booleana indicando o valor ausente
- `notnull()`: Oposto de isnull()
- `dropna()`: Retornar uma versão filtrada dos dados
- `fillna()`: Retornar uma cópia dos dados com valores ausentes preenchidos ou imputados

## Detectando valores null

As estruturas de dados do Pandas têm dois métodos úteis para detectar dados nulos: `isnull()` e `notnull()`. Qualquer um retornará uma máscara booleana sobre os dados. Por exemplo:

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

## Dropping de valores null

Além do mascaramento usado anteriormente, existem os métodos de conveniência, `dropna()` (que remove os valores de NA) e `fillna()` (que preenche os valores de NA). Para uma série, o resultado é direto:

In [16]:
data.dropna()

0        1
2    hello
dtype: object

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Não podemos descartar valores únicos de um `DataFrame`; só podemos eliminar linhas ou colunas completas. Dependendo do aplicativo, você pode querer um ou outro, então `dropna()` fornece várias opções para um `DataFrame`.

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternativa para o `dropna()`

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [22]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


## Preenchendo valores nulos

Às vezes, em vez de descartar os valores de NA, você deve substituí-los por um valor válido. Esse valor pode ser um número único como zero ou pode ser algum tipo de imputação ou interpolação dos bons valores. Você pode fazer isso no local usando o método `isnull()` como máscara, mas, como é uma operação comum, o Pandas fornece o método `fillna()`, que retorna uma cópia da matriz com os valores nulos substituídos.

Considere a seguinte série:

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Substituindo os valores Na por zero:

In [24]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Podemos especificar um preenchimento para a frente para propagar o valor anterior para a frente:

In [25]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64